In [2]:
import numpy as np
import matplotlib

In [28]:
# Constants
max_constraint = 0.17
min_constraint = -max_constraint
max_min_constraint = -0.3
min_max_constraint = -max_min_constraint
g = 9.81  # gravity
h_com = 0.8  # height of CoM
R = 1
Q = 1e6
dt = 5
N = 5  # lookahead

In [29]:
# Matrices
Pu = np.zeros((N,N))

# First column vector
for N_i in range(N):
    Pu[N_i, 0] = ((1+3*N_i+3*N_i**2)*dt**3/6 - dt * h_com/g)

for i in range(1,N):
    for j in range(1,N):
        Pu[i,j] = Pu[i-1,j-1]
print(Pu)

[[  20.42558614    0.            0.            0.            0.        ]
 [ 145.42558614   20.42558614    0.            0.            0.        ]
 [ 395.42558614  145.42558614   20.42558614    0.            0.        ]
 [ 770.42558614  395.42558614  145.42558614   20.42558614    0.        ]
 [1270.42558614  770.42558614  395.42558614  145.42558614   20.42558614]]
